In [1]:
from pathlib import Path
import os,sys
from src.ImageClassifier.Exception import CustomException
from src.ImageClassifier.loggers import logger
from src.ImageClassifier.Constants import *
from src.ImageClassifier.Utils import Save_Model,Create_Directory,read_yaml_file
from dataclasses import dataclass
import tensorflow

## Training Model 

## updating the entity file 

In [2]:
#updating the Entity file is nothing but parameter 
#which is used in yaml file we r defining as a class variable
@dataclass
class TrainingModelConfig():
    root_dir_path:Path
    trained_model_path:Path
    update_base_model_path:Path
    training_data_dir:Path #taking value from previous entity
    include_top:bool
    weights:str
    input_shape:list
    classes:int
    batch_size:int
    epochs:int
    augmentation:bool

#during training the model logs file storing into log_directory and model stroing into checkpoint model
#so we r using callback entity
@dataclass
class CallBackModelConfig():
    # define the parameters
    root_dir_path:Path
    tensorboard_log_dir_path:Path
    model_checkpoint_path:Path

## step4) update the configuration manager file

In [3]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILEPATH, param_filepath=PARAM_FILEPATH):
        # Reading the YAML files and returning a ConfigBox dictionary
        self.config = read_yaml_file(config_filepath)
        self.param = read_yaml_file(param_filepath)

        # Create artifacts directory
        Create_Directory([self.config.artifacts_root])  # This will create the artifacts root directory

    # Method to initialize the callback configuration and return the CallBackModelConfig object
    def get_callback_config(self) -> CallBackModelConfig:
        try:
            # Accessing the prepare_callback_model section of the config
            callback_config = self.config.prepare_callback_model

            # Creating necessary directories for callback artifacts
            chkpt_directory = os.path.dirname(callback_config.model_checkpoint_path)
            Create_Directory([callback_config.root_dir_path, chkpt_directory])

            # Creating and returning the CallBackModelConfig object
            call_back_config =  CallBackModelConfig(
                root_dir_path=callback_config.root_dir_path,
                tensorboard_log_dir_path=callback_config.tensorboard_log_dir_path,
                model_checkpoint_path=callback_config.model_checkpoint_path
            )
            return call_back_config

        except KeyError as e:
            raise CustomException(f"KeyError in callback config: {e}", sys)
        except Exception as e:
            raise CustomException(f"An error occurred in callback config: {e}", sys)

    # Method to initialize the training configuration and return the TrainingModelConfig object
    def get_training_config(self) -> TrainingModelConfig:
        try:
            # Accessing the prepare_training section of the config
            training_config = self.config.prepare_training

            # Creating the necessary directories for training artifacts
            Create_Directory([training_config.root_dir_path])

            # Creating and returning the TrainingModelConfig object
            return TrainingModelConfig(
                root_dir_path=training_config.root_dir_path,
                trained_model_path=training_config.trained_model_path,
                update_base_model_path=self.config.prepare_base_model.update_base_model_path,
                include_top=self.param.include_top,
                weights=self.param.weights,
                input_shape=self.param.input_shape,
                classes=self.param.classes,
                batch_size=self.param.batch_size,
                epochs=self.param.epochs,
                training_data_dir=training_config.training_data_dir,
                augmentation=self.param.augmentation
            )

        except KeyError as e:
            raise CustomException(f"KeyError in training config: {e}", sys)
        except Exception as e:
            raise CustomException(f"An error occurred in training config: {e}", sys)



In [4]:
from datetime import datetime

## step5) updating the compoents

In [5]:
#in component file we gonna initialize object to CallBackModelConfig class 
#and accessing all the parameter and value init

class CallBackModel():
    def __init__(self,config:CallBackModelConfig):
        self.config = config


    #create method to get tensorboard log directory
    @property
    def _get_tensorboard_log_dir(self):
        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

        tensorboard_log_dir = os.path.join(self.config.tensorboard_log_dir_path,f"TB_{timestamp}.log")

        # Ensure the directory exists
        os.makedirs(os.path.dirname(tensorboard_log_dir), exist_ok=True)

        tb_directory = tensorflow.keras.callbacks.TensorBoard(
            log_dir=tensorboard_log_dir
        )
        return tb_directory
    
    #method to create model_checkpoint to save best model file after training done
    @property
    def _get_checkpont_model(self):
        checkpoint_model = tensorflow.keras.callbacks.ModelCheckpoint(
            filepath=self.config.model_checkpoint_path,
            verbose=1,
            save_best_only=True
        )
        return checkpoint_model



    def callback(self):
        return[
        self._get_tensorboard_log_dir,
        self._get_checkpont_model
        ]

In [6]:
# CREATING TRAINING COMPONENT

class TrainingModel():
    def __init__(self,config:TrainingModelConfig):
        self.config = config

    #creating method for training the model first loading the sequential updated model by using model load class of tf
    def load_sequential_model(self):
        self.model = tensorflow.keras.models.load_model(
            filepath=self.config.update_base_model_path
        )

    #creating method to get train_valid_generator
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
                rescale = 1./255, #normalizing the image to pixel 0 to 255
                validation_split=0.20  #testing data would be 20%
            )

        dataflow_kwargs = dict(
                target_size=self.config.input_shape[:-1], #list [224,224,3] of image se removing the channel of image
                batch_size=self.config.batch_size,
                interpolation="bilinear" # This is the method used for resizing images.
                #Bilinear interpolation is one of the options for smooth resizing.
            )

        #calling the class of tensorflow to start the preprocessing the image and generate the image data generator
        valid_datagenerator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        ) 

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data_dir,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        #Training Data Generator with Augmentation

        if self.config.augmentation:
            train_datagenerator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator = valid_datagenerator #rtn train_datagenrator as it is without performing the augmentation

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data_dir,
            subset="training", #This loads the training data (80% of the total data).
            shuffle=True, #Training data is shuffled to prevent the model from learning the order of images rather than the content.
            **dataflow_kwargs #As with the validation generator, the image size, batch size, and interpolation method are applied here.
        )

    
    #initiate the training the model
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        #now saving the model in trained path
        Save_Model(filepath=Path(self.config.trained_model_path),model=self.model)

In [7]:
os.chdir('../')
%pwd

'd:\\DL MODEL\\ImageClassificationModel'

In [8]:
try:
    #creating an object of ConfigurationManager class
    cm = ConfigurationManager()

    #calling the method of class
    call_back_config = cm.get_callback_config()

    #calling the method to get training config from configurationmanager class
    train_config = cm.get_training_config()

    #creating an object of callbacks class
    cb = CallBackModel(config=call_back_config)

    #calling teh mthod of callbacks class
    lst_obj = cb.callback()

    #creating an object of TrainingModel
    tm = TrainingModel(config=train_config)

    #calling the method of TrainingModel class
    tm.load_sequential_model()

    #calling the valid generato method
    tm.train_valid_generator()

    tm.train(callback_list=lst_obj)

except Exception as e:
    raise CustomException(e,sys)


[2024-09-11 12:36:19,876]-INFO-66-Reading the YAML file config\config.yaml
[2024-09-11 12:36:19,883]-INFO-69-YAML file read successfully: config\config.yaml
[2024-09-11 12:36:19,885]-INFO-66-Reading the YAML file params.yaml
[2024-09-11 12:36:19,890]-INFO-69-YAML file read successfully: params.yaml
[2024-09-11 12:36:19,892]-INFO-80-Creating Directory
[2024-09-11 12:36:19,895]-INFO-80-Creating Directory
[2024-09-11 12:36:19,898]-INFO-80-Creating Directory
Found 9 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Epoch 1/10
7/7 [==============================] - ETA: 0s - loss: 0.7219 - accuracy: 0.4839
Epoch 1: val_loss improved from inf to 0.66656, saving model to artifacts/callback_model/model_checkpoint_path\model.h5
7/7 [==============================] - 10s 1s/step - loss: 0.7219 - accuracy: 0.4839 - val_loss: 0.6666 - val_accuracy: 0.8000
Epoch 2/10


d:\DL MODEL\ImageClassificationModel\cnnmodel\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - ETA: 0s - loss: 0.6977 - accuracy: 0.5484
Epoch 2: val_loss improved from 0.66656 to 0.66109, saving model to artifacts/callback_model/model_checkpoint_path\model.h5
7/7 [==============================] - 9s 1s/step - loss: 0.6977 - accuracy: 0.5484 - val_loss: 0.6611 - val_accuracy: 0.8000
Epoch 3/10
7/7 [==============================] - ETA: 0s - loss: 0.6754 - accuracy: 0.6286
Epoch 3: val_loss did not improve from 0.66109
7/7 [==============================] - 16s 2s/step - loss: 0.6754 - accuracy: 0.6286 - val_loss: 0.6612 - val_accuracy: 0.8000
Epoch 4/10
7/7 [==============================] - ETA: 0s - loss: 0.6925 - accuracy: 0.5161
Epoch 4: val_loss did not improve from 0.66109
7/7 [==============================] - 10s 1s/step - loss: 0.6925 - accuracy: 0.5161 - val_loss: 0.6646 - val_accuracy: 0.8000
Epoch 5/10
7/7 [==============================] - ETA: 0s - loss: 0.6540 - accuracy: 0.5806
Epoch 5: val_loss did not improve from 0.6610